In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#data import
train=pd.read_csv("G:\\내 드라이브\\Lab\code\\workplace\\Study\\DL\\TabNet\\data\\atc\\pivot.csv")
print(train)

     jobno   CR1_mean  CR1_min  CR1_max    CR1_std     CR1_var   ER1_mean  \
0        1  44.460927      6.0    245.0  15.831372  250.632326  46.216444   
1        2  58.524277     28.0     92.0  20.107806  404.323868  49.544407   
2        4  50.807634     25.0     88.0  18.928483  358.287479  42.368451   
3        5  45.077235     24.0     56.0   8.947864   80.064275  45.367722   
4        6  55.632718     25.0     86.0  17.811346  317.244039  48.017112   
..     ...        ...      ...      ...        ...         ...        ...   
112    137  53.955195     23.0     86.0  19.814728  392.623441  57.329167   
113    138  55.753194     23.0     95.0  21.557853  464.741016  52.681535   
114    139  46.465132     22.0     94.0  16.210828  262.790941  43.477958   
115    140  57.077212     24.0    157.0  21.608106  466.910264  53.576627   
116    141  42.721661     24.0     57.0   8.754629   76.643532  47.296829   

     ER1_min  ER1_max    ER1_std  ...  ST1_min  ST1_max  ST1_std  ST1_var  

In [3]:
encoder=LabelEncoder()
categorical_dims={}

for col in train.columns:
    train[col]=encoder.fit_transform(train[col].values)
    categorical_dims[col]=train[col].nunique()
print(categorical_dims)

{'jobno': 117, 'CR1_mean': 116, 'CR1_min': 27, 'CR1_max': 41, 'CR1_std': 116, 'CR1_var': 116, 'ER1_mean': 116, 'ER1_min': 8, 'ER1_max': 8, 'ER1_std': 116, 'ER1_var': 116, 'ST1_mean': 3, 'ST1_min': 3, 'ST1_max': 2, 'ST1_std': 2, 'ST1_var': 2, 'ST2_mean': 4, 'ST2_min': 3, 'ST2_max': 1, 'ST2_std': 4, 'ST2_var': 4, 'prod_grade': 4}


In [4]:
features=train.columns.tolist()
features.remove('prod_grade')
print(len(features))
print(features)
cat_idxs=[]
for i,x in enumerate(features):
    cat_idxs.append(i)


cat_dims=[]
for v in categorical_dims.values():
    cat_dims.append(v)

cat_dims=cat_dims[:-1]
print(cat_dims)
print(len(cat_dims))

21
['jobno', 'CR1_mean', 'CR1_min', 'CR1_max', 'CR1_std', 'CR1_var', 'ER1_mean', 'ER1_min', 'ER1_max', 'ER1_std', 'ER1_var', 'ST1_mean', 'ST1_min', 'ST1_max', 'ST1_std', 'ST1_var', 'ST2_mean', 'ST2_min', 'ST2_max', 'ST2_std', 'ST2_var']
[117, 116, 27, 41, 116, 116, 116, 8, 8, 116, 116, 3, 3, 2, 2, 2, 4, 3, 1, 4, 4]
21


In [5]:
tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetClassifier(**tabnet_params
                      )

Device used : cuda


In [6]:
from sklearn.model_selection import train_test_split

X=train.drop(['prod_grade'],axis=1)
Y=train['prod_grade']
print(Y)

X=X.to_numpy()
Y=Y.to_numpy()

X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.1, shuffle=True, random_state=1004)
X_train, X_val, y_train, y_val=train_test_split(X_train,y_train,test_size=0.1, shuffle=True, random_state=1004)

print(X_train)

0      0
1      1
2      3
3      1
4      0
      ..
112    0
113    0
114    0
115    0
116    0
Name: prod_grade, Length: 117, dtype: int32
[[ 91  72  13 ...   0   0   0]
 [ 98  60  14 ...   0   0   0]
 [ 81  32   9 ...   0   0   0]
 ...
 [ 92 115  17 ...   0   0   0]
 [ 74   6   6 ...   0   0   0]
 [  3  40  22 ...   0   2   2]]


In [7]:
max_epochs = 100 if not os.getenv("CI", False) else 2

In [8]:
save_history = []
for _ in range(2):
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['accuracy'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False,
    )
    save_history.append(clf.history["valid_accuracy"])
    
#assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 2.1892  | train_accuracy: 0.32979 | valid_accuracy: 0.36364 |  0:00:01s
epoch 1  | loss: 1.71763 | train_accuracy: 0.37234 | valid_accuracy: 0.45455 |  0:00:01s
epoch 2  | loss: 1.59906 | train_accuracy: 0.32979 | valid_accuracy: 0.45455 |  0:00:01s
epoch 3  | loss: 1.41191 | train_accuracy: 0.32979 | valid_accuracy: 0.27273 |  0:00:01s
epoch 4  | loss: 1.20379 | train_accuracy: 0.31915 | valid_accuracy: 0.18182 |  0:00:01s
epoch 5  | loss: 1.24391 | train_accuracy: 0.37234 | valid_accuracy: 0.36364 |  0:00:01s
epoch 6  | loss: 1.15698 | train_accuracy: 0.40426 | valid_accuracy: 0.27273 |  0:00:01s
epoch 7  | loss: 1.01449 | train_accuracy: 0.45745 | valid_accuracy: 0.36364 |  0:00:01s
epoch 8  | loss: 0.94755 | train_accuracy: 0.51064 | valid_accuracy: 0.45455 |  0:00:01s
epoch 9  | loss: 0.87232 | train_accuracy: 0.44681 | valid_accuracy: 0.36364 |  0:00:01s
epoch 10 | loss: 0.85156 | train_accuracy: 0.45745 | valid_accuracy: 0.36364 |  0:00:01s
epoch 11 | loss: 0.88

In [9]:
import inspect

inspect.getsource(TabNetClassifier)

'class TabNetClassifier(TabModel):\n    def __post_init__(self):\n        super(TabNetClassifier, self).__post_init__()\n        self._task = \'classification\'\n        self._default_loss = torch.nn.functional.cross_entropy\n        self._default_metric = \'accuracy\'\n\n    def weight_updater(self, weights):\n        """\n        Updates weights dictionary according to target_mapper.\n\n        Parameters\n        ----------\n        weights : bool or dict\n            Given weights for balancing training.\n\n        Returns\n        -------\n        bool or dict\n            Same bool if weights are bool, updated dict otherwise.\n\n        """\n        if isinstance(weights, int):\n            return weights\n        elif isinstance(weights, dict):\n            return {self.target_mapper[key]: value for key, value in weights.items()}\n        else:\n            return weights\n\n    def prepare_target(self, y):\n        return np.vectorize(self.target_mapper.get)(y)\n\n    def compu